In [1]:
!pip install pymorphy2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 70.0 MB/s eta 0:00:00


In [2]:
import pandas as pd

import torch
from transformers import AutoModelForSequenceClassification, BertTokenizerFast, Trainer, TrainingArguments

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop_words = stopwords.words("russian")

import re

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
def clean_text(text):
    # Remove URLs, numbers, and special characters
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)  # URLs
    text = re.sub(r'\d+', '', text)  # Numbers
    text = re.sub(r'[^\w\s]', '', text)  # Special characters
    return text

def preprocess_text(text):
    tokens = nltk.word_tokenize(text.lower())
    # remove stop words
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    # lemmatize
    def lemmatize(tokens):
        return [morph.parse(word)[0].normal_form for word in tokens]
    tokens = lemmatize(tokens)
    return ' '.join(tokens)

def all_preprocessing(df):
    df['text'] = df['text'].apply(clean_text)
    df['text'] = df['text'].apply(preprocess_text)
    return df

In [4]:
import wandb

wandb.login(key='b123af3ff1bc7e54569d0976c6405a5b3b6d2902')

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [5]:
df = pd.read_csv('/kaggle/input/wb_winter_24/train.csv', index_col='ID')
#df = all_preprocessing(df)
df.head()

,text,label
ID,,
0,"Брюки отличные, качественные, но к сожалению к...",0
1,"Отличный аппарат, в комплекте кабель и работет...",0
2,Супер 👍 спасибо большое,0
3,Получил быстро данные наушники! К наушникам пр...,0
4,Всё дошло в целости и сохранности),0


In [6]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

tokenizer = BertTokenizerFast.from_pretrained('MonoHime/rubert-base-cased-sentiment-new')
model = AutoModelForSequenceClassification.from_pretrained(
    'MonoHime/rubert-base-cased-sentiment-new', num_labels=2, ignore_mismatched_sizes=True  # Assuming binary classification
)
model.to(device)
model.train()
print(model)

tokenizer_config.json:   0%|          | 0.00/499 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/943 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at MonoHime/rubert-base-cased-sentiment-new and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [7]:
# Split dataset into training and validation
# Split dataset into training and validation
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].tolist(), 
    df['label'].tolist(), 
    test_size=0.05,
    shuffle=True,
    stratify=df['label'].tolist(),
    random_state=42
)
tokenizer = BertTokenizerFast.from_pretrained('MonoHime/rubert-base-cased-sentiment-new')

# Create a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors="pt"
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Prepare datasets
train_dataset, val_dataset = CustomDataset(train_texts, train_labels, tokenizer), CustomDataset(val_texts, val_labels, tokenizer)

# Define compute metrics function
def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    return {
        'accuracy': accuracy_score(p.label_ids, preds),
        'f1': f1_score(p.label_ids, preds, average='weighted'),
        'precision': precision_score(p.label_ids, preds, average='weighted'),
        'recall': recall_score(p.label_ids, preds, average='weighted')
    }

# Set training arguments
training_args = TrainingArguments(
    output_dir='/kaggle/working/results',
    num_train_epochs=10,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    warmup_ratio=0.1,
    weight_decay=0.01,
    learning_rate=3e-5,
    fp16=True,
    eval_strategy="epoch",
    save_strategy='epoch',
    save_total_limit=3,
    metric_for_best_model="f1"
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained('/kaggle/working/fine_tuned_model')
tokenizer.save_pretrained('/kaggle/working/fine_tuned_model')


/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: luizanigogosova (luezzka). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.18.3
wandb: Run data is saved locally in /kaggle/working/wandb/run-20241116_200938-jmutlllz
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run /kaggle/working/results
wandb: ⭐️ View project at https://wandb.ai/luezzka/huggingface
wandb: 🚀 View run at https://wandb.ai/luezzka/huggingface/runs/jmutlllz


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.108700,0.072939,0.977848,0.977893,0.977943,0.977848
2,0.053000,0.058042,0.986342,0.986291,0.986260,0.986342
3,0.038100,0.055875,0.985676,0.985676,0.985676,0.985676
4,0.029300,0.064933,0.986093,0.986111,0.986131,0.986093
5,0.022400,0.067354,0.985843,0.985764,0.985730,0.985843
6,0.016300,0.071453,0.985676,0.985680,0.985685,0.985676
7,0.013000,0.065576,0.985759,0.985728,0.985704,0.985759
8,0.010100,0.080142,0.985177,0.985142,0.985115,0.985177
9,0.008700,0.074151,0.986009,0.985977,0.985952,0.986009
10,0.007500,0.083222,0.985843,0.985810,0.985785,0.985843


('/kaggle/working/fine_tuned_model/tokenizer_config.json',
 '/kaggle/working/fine_tuned_model/special_tokens_map.json',
 '/kaggle/working/fine_tuned_model/vocab.txt',
 '/kaggle/working/fine_tuned_model/added_tokens.json',
 '/kaggle/working/fine_tuned_model/tokenizer.json')

In [8]:
test_df = pd.read_csv('/kaggle/input/wb_winter_24/test.csv', index_col='ID')
#test_df = all_preprocessing(test_df)

model.eval()  # Set model to evaluation mode

def predict(text):
    # Tokenize input text
    inputs = tokenizer(text, padding=True, truncation=True, max_length=128, return_tensors="pt")
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

    return predictions.item()  # Return predicted label (0 or 1 for binary classification)

test_df['label'] = test_df['text'].apply(predict)
test_df = test_df.drop(columns='text')
test_df.to_csv('/kaggle/working/test_predicted.csv')